In [2]:
import pandas as pd
import requests as r
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [112]:
payload ={
    "latitude": 53.5507,
    "longitude": 9.993,
    "hourly": 'temperature_2m,cloudcover,rain,snowfall,precipitation',
    "timezone": 'auto',
    "models": 'icon_seamless'
}

resp = r.get("https://ensemble-api.open-meteo.com/v1/ensemble",
             params=payload)

data = pd.DataFrame.from_dict(resp.json()['hourly'])
data['time'] = pd.to_datetime(data['time']).dt.tz_localize(resp.json()['timezone'])

data = data.dropna()

In [113]:
members = len(data.iloc[:, data.columns.str.contains('temperature_2m')].columns)

data['rain_prob'] = (((data.iloc[:, data.columns.str.contains(
    'rain')] >= 0.1).sum(axis=1) / members) * 100.)

data['rain_median'] = data.iloc[:, data.columns.str.contains('rain')].median(axis=1)
data['rain_mean'] = data.iloc[:, data.columns.str.contains('rain')].mean(axis=1)

data['cloud_mean'] = data.iloc[:, data.columns.str.contains('cloudcover')].mean(axis=1)

In [103]:
fig = make_subplots(rows=3,
                    cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02,
                    row_heights=[0.6, 0.3, 0.4])


for col_temp in data.columns[data.columns.str.contains('temperature')]:
    fig.add_trace(
        go.Scatter(
            x=data['time'],
            y=data[col_temp],
            mode='lines',
            name=col_temp,
            showlegend=False),
        row=1,
        col=1
    )

fig.add_trace(
    go.Bar(
        x=data['time'],
        y=data['rain_median'],
        text=data['rain_prob'],
        name='Rain',
        textposition='auto',
        showlegend=False),
    row=2,
    col=1
)

for col_cloud in data.columns[data.columns.str.contains('cloudcover')]:
    fig.add_trace(
        go.Scatter(
            x=data['time'],
            y=data[col_cloud],
            mode='markers',
            name=col_cloud,
            showlegend=False),
        row=3,
        col=1
    )

fig.add_trace(
    go.Scatter(
        x=data['time'],
        y=data['cloud_mean'],
        mode='lines',
        name="Mean cloud cover",
        showlegend=False,
        line=dict(color='black', width=4)),
    row=3,
    col=1
)

fig.update_layout(
            xaxis=dict(showgrid=True),
            yaxis=dict(showgrid=True, zeroline=True),
            height=800,
            margin={"r": 0.1, "t": 0.1, "l": 0.1, "b": 0.1},
            template='plotly_white',
        )

fig.show()